In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['figure.figsize'] = [20.0,8.0]
plt.rcParams['figure.dpi']=80

In [2]:
df = pd.read_csv("./dataset/RFR_lag_on_cust.csv")

In [3]:
#removing lags on customers from the dataset
df = df.drop(['t-7','t-14','t-1','t-2','t-9','t-8','t-6','t-15','t-21','t-13','t-20'],axis=1)

ValueError: labels ['t-7' 't-14' 't-1' 't-2' 't-9' 't-8' 't-6' 't-15' 't-21' 't-13' 't-20'] not contained in axis

In [ ]:
df.head().T


In [ ]:
lags_ordered_importance = ['7','14','1','2','9','8','6','15','21','13','20']
#using 8 lags for now, max n = 11
n_lags = 11
for i in range(n_lags):
    df['t-'+str(lags_ordered_importance[i])] = \
                df['NumberOfSales'].shift(int(lags_ordered_importance[i]))
    #dataset['t'] = series.values
    #dataset.head(20)
    #dataset = dataset[20:]
 

In [ ]:
df =df[21:]

In [ ]:
df.head().T

RANDOM FOREST REGRESSION

In [ ]:
df = df.drop(['NumberOfCustomers'], axis=1)
#df = df.drop(['Region','Events'], axis=1)
# I could leave events

In [ ]:
#for now let's drop unusable categorical vars
df = df.drop(['StoreType','AssortmentType','Events'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve

In [ ]:
import modify_dataset as md
train = md.get_fake_train(df)
test = md.get_fake_test(df)

In [ ]:
train.shape


In [ ]:
test.shape

In [ ]:
features = list(train.columns.values)
features.remove('NumberOfSales')
features.remove('StoreID')
features.remove('D_Year')
features.remove('Date')
features.remove('IsOpen')
print("features")

print(features)

In [ ]:
train[features].head().T

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model1 = RandomForestRegressor(n_estimators=4, max_depth=20, n_jobs=-1, verbose=2)

In [ ]:
# Can I choose to give it numpy arrays or pandas df or I can only pass np arrays?
model1.fit(train[features], train['NumberOfSales'])

In [ ]:
idx = model1.feature_importances_.argsort()[::-1]

In [ ]:
ax = sns.barplot(x=df[features].columns[idx], y=model1.feature_importances_[idx])
_ = plt.setp(ax.get_xticklabels(), rotation=-90)

In [ ]:
df_days = test[((df['D_Month'] == 1) | (df['D_Month'] == 2)) & (df['D_Year'] == 2018)]

days = df_days['Date'].loc[:'Date'].values
len(days)

In [ ]:
days=list(set(days))

In [ ]:
len(days)

In [ ]:
days=sorted(days)
days

In [ ]:
lags_list = ['t-14','t-7','t-1','t-2','t-9','t-8','t-6','t-15','t-21','t-13','t-20']

In [ ]:
#saving cheat test to test with perfect lags later
test_cheat = md.get_fake_test(df)

In [ ]:
df_train_test = df
df_train_test['_NumberOfSales'] = df_train_test['NumberOfSales']
df_train_test.loc[((df['D_Month'] == 1) | (df['D_Month'] == 2)) & (df['D_Year'] == 2018),\
                lags_list + ['_NumberOfCustomers']] = 0



In [ ]:
df_train_test.loc[(df_train_test['D_Year'] == 2018)].T

In [ ]:
# in order to avoid to re-run each time the whole notebook
df_train_test_ = df_train_test

store_rows_shapes = []

for d in days:
    stores_rows = df_train_test_.loc[(df_train_test_['Date'].values == d)]

    store_rows_shapes.append((stores_rows.shape, d))
    # stores_rows.head()

    lags_ordered = [7, 14, 1, 2, 9, 8, 6, 15, 21, 13, 20]
    lags_name = ['t-' + str(i) for i in lags_ordered]

    for i in range(len(lags_ordered)):
        stores_rows[lags_name[i]] = df_train_test_['_NumberOfSales'].shift(lags_ordered[i])

    # stores_pred = model.predict(stores_rows[features])
    stores_pred = model1.predict(stores_rows[features])
    # print(stores_pred[:5])
    df_stores_pred = pd.DataFrame(stores_pred, columns=['_NumberOfSales'], index=stores_rows.index)

    new_column = pd.Series(stores_pred, name='_NumberOfSales', index=stores_rows.index)
    df_train_test_.update(new_column)

In [ ]:
n = 2106
df_train_test_[n:n+50].T

In [ ]:
test = df_train_test_[((df['D_Month'] == 1) | (df['D_Month'] == 2)) & (df['D_Year'] == 2018)]

In [ ]:
print(test.shape)
test.head(20).T

In [ ]:
test['_NumberOfSales'] = model1.predict(test[features])

In [ ]:
# calculationg R2

# calculate RSS
df_RSS = test
print(df_RSS.shape)

df_RSS['sqrd_diff'] = df_RSS.apply(lambda x: (x['NumberOfSales'] - x['_NumberOfSales'])**2, axis=1)

RSS = df_RSS['sqrd_diff'].sum()
print('RSS:', RSS)
df_RSS.head()

In [ ]:
# calculate TSS

df_TSS = test
NS_mean = df_TSS['NumberOfSales'].mean()
print('NS_mean', NS_mean)

df_TSS['sqrd_diff'] = df_TSS.apply(lambda x:  (x['_NumberOfSales'] - NS_mean)**2, axis=1)
TSS = df_TSS['sqrd_diff'].sum()
print('TSS:', TSS)
df_TSS.head()

In [ ]:
# calculate R2
print("RSS:", RSS, "   TSS:", TSS)

R2 = 1 - (RSS / TSS)
print('R_2:', R2)



In [ ]:
from BIP import get_BIP_error
error1 = get_BIP_error(test)